In [538]:
# Imports
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
                     
from sklearn.ensemble import RandomForestClassifier

In [539]:
titanic_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

titanic_df.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [540]:
titanic_df = titanic_df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
test_df = test_df.drop(['Name', 'Ticket'], axis=1)

In [541]:
titanic_df['Embarked'] = titanic_df['Embarked'].fillna('S')

In [542]:
#Embarked

embarked_dummies_titanic = pd.get_dummies(titanic_df['Embarked'])
embarked_dummies_titanic.drop(['S'], axis=1, inplace=True)

embarked_dummies_test = pd.get_dummies(test_df['Embarked'])
embarked_dummies_test.drop(['S'], axis=1, inplace=True)

titanic_df = titanic_df.join(embarked_dummies_titanic)
test_df = test_df.join(embarked_dummies_test)

titanic_df.drop(['Embarked'], axis=1, inplace=True)
test_df.drop(['Embarked'], axis=1, inplace=True)

In [543]:
# Fare
#Vemos los NaN y los rellenamos con la media
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)

titanic_df['Fare'] = titanic_df['Fare'].astype(int)
test_df['Fare'] = test_df['Fare'].astype(int)

In [544]:
# Age

# Obtenemos la media
average_age_titanic = titanic_df["Age"].mean()
average_age_test   = test_df["Age"].mean()

# Dropeamos los valores nulos como tipo int
titanic_df['Age'].dropna().astype(int)

# Rellenamos los valores NaN por la media de edad de titanic y de test
titanic_df["Age"][np.isnan(titanic_df["Age"])] = average_age_titanic
test_df["Age"][np.isnan(test_df["Age"])] = average_age_test

# Convertimos de float a int
titanic_df['Age'] = titanic_df['Age'].astype(int)
test_df['Age']    = test_df['Age'].astype(int)

C:\Users\DIEGOMAESTREVIDAL\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\DIEGOMAESTREVIDAL\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [545]:
# Cabin

# Drop de los valores NaN en Cabin
titanic_df.drop("Cabin",axis=1,inplace=True)
test_df.drop("Cabin",axis=1,inplace=True)

In [546]:
# Family

#Juntamos Parch y SibSp en una nueva feature llamada Family
titanic_df['Family'] = titanic_df['Parch'] + titanic_df['SibSp']

titanic_df['Family'].loc[titanic_df['Family']>0] = 1
titanic_df['Family'].loc[titanic_df['Family']==0] = 0

test_df['Family'] = test_df['Parch'] + test_df['SibSp']
test_df['Family'].loc[test_df['Family']>0] = 1
test_df['Family'].loc[test_df['Family']==0] = 0

# Hacemos drop en SibSp y Parch
titanic_df = titanic_df.drop(['SibSp', 'Parch'], axis=1)
test_df = test_df.drop(['SibSp', 'Parch'], axis=1)

C:\Users\DIEGOMAESTREVIDAL\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [547]:
# Sex

# Clasificamos los pasajeros en hombre, mujeres y niños
def get_person(passenger):
    age,sex = passenger
    return 'child' if age < 16 else sex
    
titanic_df['Person'] = titanic_df[['Age','Sex']].apply(get_person,axis=1)
test_df['Person']    = test_df[['Age','Sex']].apply(get_person,axis=1)

# Al tener una columna person, no necesitamos usar la columna Sex
titanic_df.drop(['Sex'],axis=1,inplace=True)
test_df.drop(['Sex'],axis=1,inplace=True)

# Creamos varables para la columna persona, y hacemos drop en los hombre ya que tienen la menor tasa de sobrevivir
person_dummies_titanic = pd.get_dummies(titanic_df['Person'])
person_dummies_titanic.columns = ['Child','Female','Male']
person_dummies_titanic.drop(['Male'], axis=1, inplace=True)

person_dummies_test = pd.get_dummies(test_df['Person'])
person_dummies_test.columns = ['Child','Female','Male']
person_dummies_test.drop(['Male'], axis=1, inplace=True)

titanic_df = titanic_df.join(person_dummies_titanic)
test_df    = test_df.join(person_dummies_test)

person_perc = titanic_df[["Person", "Survived"]].groupby(['Person'],as_index=False).mean()

titanic_df.drop(['Person'],axis=1,inplace=True)
test_df.drop(['Person'],axis=1,inplace=True)

In [548]:
# Pclass

# Creamos una variable para Pclass y quitamos la columna 3 ya que es la que menor tasa de sobrevivir tiene
pclass_dummies_titanic  = pd.get_dummies(titanic_df['Pclass'])
pclass_dummies_titanic.columns = ['Class_1','Class_2','Class_3']
pclass_dummies_titanic.drop(['Class_3'], axis=1, inplace=True)

pclass_dummies_test  = pd.get_dummies(test_df['Pclass'])
pclass_dummies_test.columns = ['Class_1','Class_2','Class_3']
pclass_dummies_test.drop(['Class_3'], axis=1, inplace=True)

titanic_df.drop(['Pclass'],axis=1,inplace=True)
test_df.drop(['Pclass'],axis=1,inplace=True)

titanic_df = titanic_df.join(pclass_dummies_titanic)
test_df    = test_df.join(pclass_dummies_test)

In [549]:
# Definimos las variables para hacer el random forest

X_train = titanic_df.drop("Survived",axis=1)
Y_train = titanic_df["Survived"]
X_test  = test_df.drop("PassengerId",axis=1).copy()

In [550]:
# Random Forests

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred_Best = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)

0.96071829405162734

In [551]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred_Best
    })
submission.to_csv('titanic.csv', index=False)